<a href="https://colab.research.google.com/github/papapabi/torch-sandbox/blob/main/image_classification_preloaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image classification - Cifar10

Cifar10 consists of 10 classes of 60_000 32x32 color images in 10 classes, with 6_000 images per class. There are 50_000 training images and 10_000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

## Objective

Classify an image if it belongs to any of the 10 classes.

## Getting `torch.utils.data.Dataset`s from `torchvision`

In [131]:
import torchvision

from torchvision import transforms

In [132]:
%%capture
original_train_ds = torchvision.datasets.CIFAR10("./data", train=True, download=True, transform=transforms.ToTensor())
test_ds = torchvision.datasets.CIFAR10("./data", train=False, download=True, transform=transforms.ToTensor())
x_sample, y_sample = original_train_ds[0]

In [133]:
print(x_sample.shape)
print(y_sample)

torch.Size([3, 32, 32])
6


In [134]:
# label mapping for a torchvision.dataset
{i: label for i, label in enumerate(original_train_ds.classes)}

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

## Splitting train into train/val

To preserve the class ratios, we need to perform stratified sampling on the train set.

In [135]:
from sklearn.model_selection import train_test_split

# Get the full indices for the train dataset
original_train_indices = list(range(len(original_train_ds)))

labels_for_stratification = original_train_ds.targets

# Get the indices for train and val from the dataset
train_indices, val_indices = train_test_split(original_train_indices, test_size=0.2, random_state=42, stratify=labels_for_stratification)

In [136]:
import torch

train_ds = torch.utils.data.Subset(dataset=original_train_ds, indices=train_indices)
val_ds = torch.utils.data.Subset(dataset=original_train_ds, indices=val_indices)

In [137]:
print("\n--- Split Dataset Info ---")
print(f"Train dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")


--- Split Dataset Info ---
Train dataset size: 40000
Validation dataset size: 10000


In [138]:
train_ds[0]

(tensor([[[0.2196, 0.2314, 0.2745,  ..., 0.5412, 0.5608, 0.6157],
          [0.2157, 0.2471, 0.3176,  ..., 0.5294, 0.6706, 0.6392],
          [0.2510, 0.3137, 0.3176,  ..., 0.6471, 0.7608, 0.5725],
          ...,
          [0.1765, 0.1961, 0.2627,  ..., 0.5294, 0.4431, 0.5020],
          [0.1843, 0.1843, 0.2510,  ..., 0.4392, 0.3961, 0.4314],
          [0.1647, 0.2078, 0.2471,  ..., 0.4902, 0.4314, 0.3686]],
 
         [[0.1961, 0.2000, 0.2549,  ..., 0.6039, 0.6196, 0.6588],
          [0.1961, 0.2275, 0.3020,  ..., 0.5843, 0.7333, 0.6941],
          [0.2431, 0.2980, 0.3059,  ..., 0.7020, 0.8275, 0.6235],
          ...,
          [0.1608, 0.1804, 0.2471,  ..., 0.5294, 0.4431, 0.4980],
          [0.1765, 0.1765, 0.2431,  ..., 0.4157, 0.3725, 0.4078],
          [0.1647, 0.2039, 0.2353,  ..., 0.4471, 0.3961, 0.3412]],
 
         [[0.1647, 0.1608, 0.2000,  ..., 0.5804, 0.6000, 0.6471],
          [0.1490, 0.1843, 0.2549,  ..., 0.5569, 0.7176, 0.6941],
          [0.1843, 0.2627, 0.2706,  ...,

In [139]:
train_ds[0][1]

6

In [140]:
type(train_ds)

torch.utils.data.dataset.Subset

### Verify stratified sampling

In [141]:
import numpy as np

# --- Verification of Stratified Sampling ---

def get_label_distribution(dataset_subset):
    """
    Calculates the distribution of labels within a torch.utils.data.Subset.
    Assumes the original dataset (accessed via dataset_subset.dataset) has a .targets attribute.
    """
    labels = [dataset_subset.dataset]
    unique, counts = np.unique(labels, return_counts=True)
    total = len(labels)
    distribution = {
        label: {'count': count, 'ratio': count / total}
        for label, count in zip(unique, counts)
    }
    return distribution, total

print("\n--- Label Distributions ---")


# 2. Train Dataset Distribution
train_dist, train_total = get_label_distribution(train_ds)
print("\nTrain Dataset:")
for label, info in train_dist.items():
    print(f"  Class {label}: Count = {info['count']}, Ratio = {info['ratio']:.4f}")

# 3. Validation Dataset Distribution
val_dist, val_total = get_label_distribution(val_ds)
print("\nValidation Dataset:")
for label, info in val_dist.items():
    print(f"  Class {label}: Count = {info['count']}, Ratio = {info['ratio']:.4f}")

# --- Compare Ratios ---
print("\n--- Ratio Comparison (Train vs. Validation vs. Original) ---")
all_labels = sorted(set(original_dist.keys()).union(train_dist.keys()).union(val_dist.keys()))

for label in all_labels:
    original_ratio = original_dist.get(label, {}).get('ratio', 0)
    train_ratio = train_dist.get(label, {}).get('ratio', 0)
    val_ratio = val_dist.get(label, {}).get('ratio', 0)

    print(f"Class {label}: Original Ratio={original_ratio:.4f}, Train Ratio={train_ratio:.4f}, Val Ratio={val_ratio:.4f}")

    # Check for approximate equality (due to potential rounding or small sample sizes)
    if not np.isclose(original_ratio, train_ratio, atol=1e-2) or \
       not np.isclose(original_ratio, val_ratio, atol=1e-2):
        print(f"  WARNING: Ratios for Class {label} are not closely matched. Stratification might be imperfect due to small class counts or very small overall dataset size.")

print("\n--- Stratification Verification Complete ---")
print("If the ratios for each class are very similar across Original, Train, and Val datasets,")
print("then stratified sampling has worked correctly.")



--- Label Distributions ---


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (1, 50000, 2) + inhomogeneous part.

## Getting `torch.utils.data.DataLoaders`

In [142]:
batch_size = 128

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

## Training and evaluation code

In [148]:
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm

In [149]:
from collections.abc import Mapping, Sequence

def move(obj, device):
    """
    Recursively moves PyTorch tensors and modules within a Python object to a specified device.

    Args:
        obj: The Python object to move to a device, or to move its contents to a device.
             Can be a torch.Tensor, torch.nn.Module, list, tuple, set, dict, or other types.
        device: The compute device (e.g., 'cpu', 'cuda:0') to move objects to.

    Returns:
        The object with its PyTorch components moved to the specified device.
    """
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, torch.nn.Module):
        return obj.to(device)
    elif isinstance(obj, Mapping):
        return {k: move(v, device) for k, v in obj.items()}
    elif isinstance(obj, Sequence) and not isinstance(obj, str): # Exclude strings as they are sequences of chars
        return type(obj)(move(x, device) for x in obj)
    else:
        return obj

In [150]:
def run_epoch(model, optimizer, data_loader, loss_func, device, results, score_funcs, prefix="", desc=None, scaler=None):
    """
    Runs a single epoch of training or validation in PyTorch.

    Args:
        model (torch.nn.Module): The PyTorch model to run for one epoch.
        optimizer (torch.optim.Optimizer): The object that will update the weights of the network.
                                           Pass None if in evaluation mode (no optimization needed).
        data_loader (torch.utils.data.DataLoader): DataLoader object that returns tuples of (input, label) pairs.
        loss_func (callable): The loss function that takes in two arguments (model outputs, labels)
                              and returns a scalar loss.
        device (torch.device or str): The compute location to perform training/evaluation (e.g., 'cpu', 'cuda:0').
        results (dict): A dictionary to store epoch-wise metrics.
        score_funcs (dict): A dictionary of scoring functions (name: function) to use to evaluate
                            the performance of the model. Each function should take (y_true, y_pred).
        prefix (str): A string to prefix to any scores placed into the `results` dictionary.
                      Commonly 'train_' or 'val_'.
        desc (str, optional): A description to use for the progress bar.
        scaler: (sklearn.preprocessing.BaseScaler, optional): An optional scaler for target variables when scaling is applied during preprocessing.

    Returns:
        float: Time spent on the epoch in seconds.
    """
    running_loss = []
    y_true_all = []
    y_pred_all = []

    start = time.time()

    # Set model to training or evaluation mode
    # NOTE: Layers like Dropout and BatchNorm behave differently during training and evaluation.
    # model.train() enables their training-specific behavior (e.g., dropout randomness, batch norm updating running stats),
    # while model.eval() sets them to evaluation mode (e.g., no dropout, batch norm using learned running stats)
    if optimizer is not None:
        model.train()
        # Enable anomaly detection for debugging during training
        # with torch.autograd.set_detect_anomaly(True): # Uncomment for debugging
        #     for inputs, labels in tqdm(data_loader, desc=desc, leave=False):
        #         ...
    else:
        model.eval()


    # Use torch.no_grad() for evaluation phase to save memory and speed up
    # computations by not building the computational graph.
    with torch.no_grad() if optimizer is None else torch.enable_grad():
        for inputs, labels in tqdm(data_loader, desc=desc, leave=False):
            # Move the batch to the device we are using.
            inputs = move(inputs, device)
            labels = move(labels, device)

            # Forward pass
            y_hat = model(inputs)

            # Compute loss
            loss = loss_func(y_hat, labels)

            if model.training:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            # Use .item() for scalar loss to prevent memory leaks in the computational graph
            running_loss.append(loss.item())

            # Collect predictions and true labels for metric calculation
            # It's generally good practice to keep these on CPU for metric calculation
            # and to convert to numpy for compatibility with libraries like scikit-learn.
            # Only process if score_funcs are provided and labels are tensors.
            if len(score_funcs) > 0 and isinstance(labels, torch.Tensor):
                # Detach from graph, move to CPU, convert to numpy
                # For classification, often want raw logits for metrics,
                # then apply argmax for accuracy.
                # For regression, y_hat is already numerical.
                labels_np = labels.detach().cpu().numpy()
                y_hat_np = y_hat.detach().cpu().numpy()

                # Extend with current batch's data
                y_true_all.extend(labels_np.tolist())
                y_pred_all.extend(y_hat_np.tolist())

    # End training/evaluation epoch
    end = time.time()

    # Post-epoch metric calculations
    y_pred_final = np.asarray(y_pred_all)
    y_true_final = np.asarray(y_true_all) # Ensure y_true is also a numpy array

    # Handle classification output (e.g., logits to class predictions)
    # This logic assumes `y_pred_all` contains raw model outputs (logits or probabilities)
    # and `y_true_all` contains integer class labels for classification, or continuous values for regression.
    if y_pred_final.size > 0 and len(y_pred_final.shape) == 2 and y_pred_final.shape[1] > 1:
        # Assuming multi-class classification where y_hat are logits/probabilities
        y_pred_final_processed = np.argmax(y_pred_final, axis=1)
    else:
        # Assume regression or binary classification (where y_hat might be a single value)
        y_pred_final_processed = y_pred_final

    # Store results
    results[prefix + " loss"].append(np.mean(running_loss))

    if scaler:
        y_true_final = scaler.inverse_transform(y_true_final)
        y_pred_final_processed = scaler.inverse_transform(y_pred_final_processed)

    for name, score_func in score_funcs.items():
        try:
            # Pass the processed predictions and true labels to score functions
            results[prefix + " " + name].append(score_func(y_true_final, y_pred_final_processed))
        except Exception as e: # Catch specific exception or general Exception
            print(f"Warning: Error calculating score '{name}': {e}. Appending NaN.")
            results[prefix + " " + name].append(float("NaN"))

    return end - start # time spent on epoch

In [151]:
import os

from typing import Callable, Optional, Union, Dict, Any
from collections import defaultdict

def train_network(
    model: nn.Module,
    loss_func: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    train_loader: DataLoader,
    val_loader: Optional[DataLoader] = None,
    score_funcs: Optional[Dict[str, Callable[[Any, Any], float]]] = None,
    epochs: int = 50,
    device: Union[str, torch.device] = "cpu",
    checkpoint_file: Optional[str] = None,
    lr_schedule: Optional[torch.optim.lr_scheduler._LRScheduler] = None,
    optimizer: Optional[torch.optim.Optimizer] = None,
    disable_tqdm: bool = False,
    scaler=None,
) -> pd.DataFrame:
    """
    Trains a PyTorch neural network.

    Args:
        model: The PyTorch model to train.
        loss_func: The loss function that takes in model outputs and labels, and returns a scalar loss.
        train_loader: PyTorch DataLoader for training data.
        val_loader: Optional PyTorch DataLoader for validation data, evaluated after each epoch.
        score_funcs: A dictionary of scoring functions (name: function) to evaluate model performance.
                     Each function should take (y_true, y_pred).
        epochs: The number of training epochs to perform.
        device: The compute location (e.g., 'cpu', 'cuda:0') to perform training.
        checkpoint_file: Optional path to a file for saving/loading model checkpoints.
        lr_schedule: The learning rate scheduler. If provided, `optimizer` must also be provided.
        optimizer: The optimizer used to update model parameters. If None, AdamW is used by default.
        disable_tqdm: If True, disables the progress bar.
        scaler: (sklearn.preprocessing.BaseScaler, optional): An optional scaler for target variables when scaling is applied during preprocessing.

    Returns:
        pd.DataFrame: A DataFrame containing epoch-wise training, validation, and test results.
    """
    if score_funcs is None:
        score_funcs = {}

    # Initialize results dictionary using defaultdict for cleaner appending
    results: Dict[str, list] = defaultdict(list)

    # Move model to the specified device
    model.to(device)

    if optimizer is None:
        print("Optimizer not provided. Using AdamW as default.")
        optimizer = torch.optim.AdamW(model.parameters())

    start_epoch = 0
    # Load from checkpoint if specified and exists
    if checkpoint_file and os.path.exists(checkpoint_file):
        print(f"Loading checkpoint from {checkpoint_file}...")
        checkpoint = torch.load(checkpoint_file, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        # Restore results if available
        if 'results' in checkpoint:
            # Ensure loaded results are compatible with defaultdict if needed
            for k, v in checkpoint['results'].items():
                results[k] = v
        print(f"Resuming training from epoch {start_epoch}.")
    elif checkpoint_file and not os.path.exists(checkpoint_file):
        print(f"Checkpoint file '{checkpoint_file}' not found. Starting training from scratch.")

    total_train_time = sum(results.get("total time", [0.0])) # Accumulate time if resuming, initialize as float

    # Main training loop
    for epoch in tqdm(range(start_epoch, epochs), desc="Overall Epoch", disable=disable_tqdm):
        # --- Training Phase ---
        print(f"\nEpoch {epoch+1}/{epochs} - Training...")
        current_epoch_train_time = run_epoch(
            model, optimizer, train_loader, loss_func, device,
            results, score_funcs, prefix="train", desc="Training Batch", scaler=scaler,
        )
        total_train_time += current_epoch_train_time

        results["epoch"].append(epoch)
        results["total time"].append(total_train_time)

        # --- Validation Phase ---
        if val_loader is not None:
            print(f"Epoch {epoch+1}/{epochs} - Validating...")
            # optimizer=None ensures run_epoch is in evaluation mode
            run_epoch(
                model, None, val_loader, loss_func, device,
                results, score_funcs, prefix="val", desc="Validation Batch", scaler=scaler,
            )

        # --- Learning Rate Schedule Step ---
        if lr_schedule is not None:
            if isinstance(lr_schedule, torch.optim.lr_scheduler.ReduceLROnPlateau):
                # ReduceLROnPlateau needs a metric; typically validation loss
                if val_loader is not None and "val loss" in results and len(results["val loss"]) > 0:
                    lr_schedule.step(results["val loss"][-1])
                else:
                    print("Warning: ReduceLROnPlateau scheduler requires validation loss but 'val_loader' is None or 'val loss' not found. Skipping step.")
            else:
                lr_schedule.step()
            print(f"Learning rate adjusted to: {optimizer.param_groups[0]['lr']:.6f}")

        # --- Checkpointing ---
        if checkpoint_file is not None:
            try:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'lr_scheduler_state_dict': lr_schedule.state_dict() if lr_schedule else None,
                    'results': dict(results) # Convert defaultdict to dict for saving
                }, checkpoint_file)
                print(f"Checkpoint saved to {checkpoint_file} at end of epoch {epoch+1}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    print("\nTraining complete!")
    return pd.DataFrame.from_dict(results)

In [152]:
def eval_network(
    model: nn.Module,
    loss_func: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    test_loader: DataLoader,
    score_funcs: Optional[Dict[str, Callable[[Any, Any], float]]] = None,
    device: Union[str, torch.device] = "cpu",
    checkpoint_file: Optional[str] = None,
    disable_tqdm: bool = False,
    scaler=None,
) -> Dict[str, float]:
    """
    Evaluates a PyTorch neural network on a held-out test set.

    Args:
        model: The PyTorch model to evaluate.
        loss_func: The loss function that takes in model outputs and labels, and returns a scalar loss.
        test_loader: PyTorch DataLoader for the test data.
        score_funcs: A dictionary of scoring functions (name: function) to evaluate model performance.
                     Each function should take (y_true, y_pred).
        device: The compute location (e.g., 'cpu', 'cuda:0') to perform evaluation.
        checkpoint_file: Optional path to a file for loading a pre-trained model checkpoint.
        disable_tqdm: If True, disables the progress bar.

    Returns:
        Dict[str, float]: A dictionary containing the evaluation loss and scores.
    """
    if score_funcs is None:
        score_funcs = {}

    # Move model to the specified device
    model.to(device)

    # Load from checkpoint if specified and exists
    if checkpoint_file and os.path.exists(checkpoint_file):
        print(f"Loading model from checkpoint: {checkpoint_file} for evaluation...")
        checkpoint = torch.load(checkpoint_file, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Model loaded successfully.")
    elif checkpoint_file and not os.path.exists(checkpoint_file):
        print(f"Warning: Checkpoint file '{checkpoint_file}' not found. Evaluating with current model state.")

    # Set model to evaluation mode
    model.eval()

    running_loss = []
    y_true_all = []
    y_pred_all = []

    start = time.time()

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating Test Set", leave=False, disable=disable_tqdm):
            inputs = move(inputs, device)
            labels = move(labels, device)

            y_hat = model(inputs)
            loss = loss_func(y_hat, labels)
            running_loss.append(loss.item())

            if len(score_funcs) > 0 and isinstance(labels, torch.Tensor):
                labels_np = labels.detach().cpu().numpy()
                y_hat_np = y_hat.detach().cpu().numpy()

                y_true_all.extend(labels_np.tolist())
                y_pred_all.extend(y_hat_np.tolist())

    end = time.time()
    eval_time = end - start

    # Post-evaluation metric calculations
    y_pred_final = np.asarray(y_pred_all)
    y_true_final = np.asarray(y_true_all)

    if y_pred_final.size > 0 and len(y_pred_final.shape) == 2 and y_pred_final.shape[1] > 1:
        y_pred_final_processed = np.argmax(y_pred_final, axis=1)
    else:
        y_pred_final_processed = y_pred_final

    final_results: Dict[str, float] = {}
    final_results["test loss"] = np.mean(running_loss)
    final_results["test_eval_time_seconds"] = eval_time

    if scaler:
        y_true_final = scaler.inverse_transform(y_true_final)
        y_pred_final_processed = scaler.inverse_transform(y_pred_final_processed)

    for name, score_func in score_funcs.items():
        try:
            final_results[f"test {name}"] = score_func(y_true_final, y_pred_final_processed)
        except Exception as e:
            print(f"Warning: Error calculating score '{name}' during test evaluation: {e}. Setting to NaN.")
            final_results[f"test {name}"] = float("NaN")

    print(f"\nTest Evaluation Complete. Time: {eval_time:.2f} seconds.")
    print("Test Results:")
    for metric, value in final_results.items():
        print(f"  {metric}: {value:.4f}")

    return final_results

## Defining the model

In [154]:
import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

D = 19 # we have 19 features
n = 32
target_classes = 10

loss_func = torch.nn.CrossEntropyLoss()
score_funcs = {"accuracy": accuracy_score, "precision": precision_score, "recall": recall_score, "f1": f1_score}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [156]:
class ResidualBottleNeck(torch.nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=3, leak_rate=0.1):
        super().__init__()
        # how much padding will our convolutional layers need to maintain input shape
        pad = (kernel_size - 1) // 2
        # the bottleneck should be smaller, so output/4 or input
        bottleneck = max(out_channels//4, in_channels)

        # Defines (3) sets of BN and convolution layers that we need
        # For 1x1 convs, we use padding=0 because 1x1 will not change shape
        self.F = torch.nn.Sequential(
            torch.nn.BatchNorm2d(in_channels),
            torch.nn.LeakyReLU(leak_rate),
            torch.nn.Conv2d(in_channels, bottleneck, 1, padding=0),

            torch.nn.BatchNorm2d(bottleneck),
            torch.nn.LeakyReLU(leak_rate),
            torch.nn.Conv2d(bottleneck, bottleneck, kernel_size, padding=pad),

            torch.nn.BatchNorm2d(bottleneck),
            torch.nn.LeakyReLU(leak_rate),
            torch.nn.Conv2d(bottleneck, out_channels, 1, padding=0)
        )

        # by default, our shortcut is the identity function
        self.shortcut = torch.nn.Identity()

        if in_channels != out_channels:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels, 1, padding=0),
                torch.nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        return self.shortcut(x) + self.F(x)